In [1]:
import zipfile
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pathlib

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling, BatchNormalization, GlobalAveragePooling2D
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.optimizers import Adam

In [2]:
# Download zip file dari drive
!gdown --id 143TYYJi90wjauEax5z8vnAPmNnn5BHHj

zip_file_path = '/content/Prakitkum_NIM_Ganjil.zip'
extract_path = '/content/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Extrak ZIP File
extracted_folder = os.listdir(extract_path)
print(extracted_folder)


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=143TYYJi90wjauEax5z8vnAPmNnn5BHHj
To: /content/Prakitkum_NIM_Genap.zip
100% 2.10M/2.10M [00:00<00:00, 27.8MB/s]


FileNotFoundError: [Errno 2] No such file or directory: '/content/Prakitkum_NIM_Ganjil.zip'

In [ ]:
# Define image size and batch size
tf.random.set_seed(42)
np.random.seed(42)

IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
NUM_CLASSES = 5
CLASSES = ["Boxer", "Dachshund", "Golden_Retriever", "Poodle", "Rootweiler"]
DATASET_PATH = "/content/Prakitkum_NIM_Ganjil"

In [ ]:
data_dir = pathlib.Path(DATASET_PATH)

In [ ]:
# Fungsi Latih dan Test dataset
train_generator = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=234,
    image_size= (IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)

validation_generator = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=234,
    image_size= (IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_generator.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(CLASSES[labels[i]])
    plt.axis("off")

In [ ]:
# # Membangun Model CNN untuk gambar greyscale
# model = Sequential([

#     Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

#     # Layer Pertama
#     Conv2D(32, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     MaxPooling2D(pool_size=(2, 2)),

#     # Layer Kedua
#     Conv2D(64, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     MaxPooling2D(pool_size=(2, 2)),

#     # Layer Ketiga
#     Conv2D(128, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     MaxPooling2D(pool_size=(2, 2)),

#     # Layer Keempat
#     Conv2D(128, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     MaxPooling2D(pool_size=(2, 2)),

#     # Layer Kelima
#     Conv2D(256, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     MaxPooling2D(pool_size=(2, 2)),

#     # Menggunakan GlobalAveragePooling2D untuk mengurangi dimensi
#     GlobalAveragePooling2D(),

#     # Fully connected layers
#     Flatten(),
#     Dense(4096, activation='relu'),
#     Dropout(0.5),
#     Dense(4096, activation='relu'),
#     Dropout(0.5),
#     Dense(NUM_CLASSES, activation='softmax')  # NUM_CLASSES = jumlah kelas yang akan diprediksi
# ])


model = Sequential([

    Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),  # input_shape untuk greyscale (1 channel)

    # Layer Pertama
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    # Layer Kedua
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    # Layer Ketiga
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    # Menggunakan GlobalAveragePooling2D untuk mengurangi dimensi
    GlobalAveragePooling2D(),

    # Fully connected layers
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Latih Model
history = model.fit(train_generator,epochs=100, batch_size=BATCH_SIZE, validation_data=validation_generator)

In [ ]:
# Memplot akurasi dan loss pelatihan dan validasi
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print('Test accuracy :', accuracy)
print('Test loss :', loss)

In [ ]:
y_true = []
y_pred = []

for images, labels in validation_generator:
    batch_predictions = model.predict(images)
    batch_pred_labels = np.argmax(batch_predictions, axis=1)
    y_true.extend(labels.numpy())
    y_pred.extend(batch_pred_labels)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

In [ ]:
print(classification_report(y_true, y_pred, target_names=CLASSES))

In [ ]:
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CLASSES)
disp.plot()
plt.show()